# Ex 3. Separate inhibitory population

In [3]:
import numpy as np
import matplotlib.pyplot as plt

**3.1.** Write down the total input to an excitatory and an inhibitory neuron. Show that the average input
to an excitatory neuron is equivalent to the input to a neuron in the model of exercise 2, for b = 0.

The total input to an excitatory neuron is comprised of inputs from inhibitory and excitatory neurons:
$$h_{i} = h_{exc} - h_{inh}$$
Firstly, we consider $h_{exc}$:
\begin{align}
h_{exc} = W \sigma_{exc}
 = \sum_{j}^{N} W_{ij}^{E \leftarrow E} \sigma_{j}(t)
 = \sum_{j}^{N} \frac{c}{N} \sum_{\mu}^{M} \xi_{i}^{\mu} \xi_{j}^{\mu} \sigma_{j}(t)
\end{align}
Secondly, we consider $h_{inh}$:
\begin{align}
h_{inh} = W \sigma_{inh}
 = \sum_{k}^{K} W_{ik}^{E \leftarrow I} \sigma_{k}^{I}(t)
 = \sum_{k}^{K} \frac{ca}{N_{I}} \sum_{\mu}^{M} \xi_{i}^{\mu} \sigma_{k}^{I}(t)
\end{align}
Hence, the total input to the excitatory neuron is:
\begin{align}
h_{i} = \sum_{j}^{N} W_{ij}^{E \leftarrow E} \sigma_{j}(t) - \sum_{k}^{N_{I}} W_{ik}^{E \leftarrow I} \sigma_{k}^{I}(t)
\end{align}

Analogously, the total input to the inhibitory neuron is equal to:
\begin{align}
h_{k} = \sum_{k} W_{ki}^{I \leftarrow E} \sigma_{k}(t) = \frac{1}{K} \sum_{k} \sigma_{k}(t),
\end{align}
where $k \in S$, where $S$ is the set of pre-synaptic neurons.

We assume that the average input from both excitatory and inhibitory neurons into excitatory neurons is independent and identically distributed and equal to $a$:
\begin{align}
<\sigma_{j}(t)> = <\sigma_{k}^{I}(t)> = a
\end{align}
Theredfore, we can re-write the total input to excitatory neuron equation:
\begin{align}
h_{i}(t) = \sum_{j}^{N} \frac{c}{N} \sum_{\mu}^{M} \xi_{i}^{\mu} \xi_{j}^{\mu} a - \sum_{k}^{K} \frac{ca}{N_{I}} \sum_{\mu}^{M} \xi_{i}^{\mu} a
= \sum_{j}^{N} \frac{ca}{N} \sum_{\mu}^{M} \xi_{i}^{\mu} \xi_{j}^{\mu} - \sum_{k}^{K} \frac{ca^{2}}{N_{I}} \sum_{\mu}^{M} \xi_{i}^{\mu}
\end{align}


\begin{align}
h_{i}(t) = \frac{ca}{N} \sum_{\mu}^{M} \xi_{i}^{\mu} \sum_{j}^{N} \xi_{j}^{\mu} - \sum_{k}^{K} \frac{ca^{2}}{N_{I}} \sum_{\mu}^{M} \xi_{i}^{\mu}
\end{align}
$$ = \frac{ca}{N} \sum_{\mu}^{M} \xi_{i}^{\mu} N a - \sum_{k}^{K} \frac{ca^{2}}{N_{I}} \sum_{\mu}^{M} \xi_{i}^{\mu}$$

## -> correct this, solution on paper

**3.2.** Write a method for simulating this new model.
There are two ways of updating the states. Either the input hi(t) to excitatory neuron i depends on σI(t)
(synchronous update); or hi(t) depends on σI(t + 1) (sequential update). Implement both.


In [ ]:
N=300
N_I=80

def generate_low_activity_patterns(M, N, a):
    """
    Generate low-activity random patterns with specified activity level.
    """
    return np.random.choice([0, 1], size=(M, N), p=[1-a, a])

def stochastic_spike_variable_exc(state):
        firing_probability = 0.5 * (state + 1) 
        sigma = np.random.binomial(1, firing_probability) # P{σ_i(t) = +1 | S_i(t)}
        return sigma

def stochastic_spike_variable_inh(K, sigma):
        firing_probability = 1/K * np.sum(sigma)
        sigma = np.random.binomial(1, firing_probability) # P{σ_i(t) = +1 | S_i(t)}
        return sigma
'''
def compute_overlap_exc(patterns, sigma, sigma_I, a, N, N_I):
    """
    Compute synaptic weights for excitatory-excitatory.
    """
    M = patterns.shape[0]
    c = 2 / (a * (1 - a))
    m = np.zeros(patterns.shape[0])
    for i in range(len(m)):
        m[i] = c / N * np.dot(patterns[i, :],sigma) - c / N_I * a[i] * np.sum(sigma_I)
    return m

def compute_syn_weights_inh(K, N_I):
    """
    Compute synaptic weights for inhibitory-excitatory.
    """
    return np.ones(N_I)/K

def compute_inputs_exc(m, patterns):
    """
    Compute the state of the network.
    """
    return np.dot(m, patterns.T)

def compute_inputs_inh(K, N_I, sigma):
    """
    Compute the state of the network.
    """
    return compute_syn_weights_inh(K, N_I)*np.sum(sigma)
'''

In [54]:
def generate_low_activity_patterns(M, N, a):
    """
    Generate low-activity random patterns with specified activity level.
    """
    return np.random.choice([0, 1], size=(M, N), p=[1-a, a])

def stochastic_spike_variable_exc(state):
        firing_probability = 0.5 * (state + 1) 
        sigma = np.random.binomial(1, firing_probability) # P{σ_i(t) = +1 | S_i(t)}
        return sigma

def stochastic_spike_variable_inh(h_inh):
        h_inh = np.tanh(h_inh)
        sigma_I = np.random.binomial(1, h_inh) # P{σ_i(t) = +1 | S_i(t)}
        return sigma_I

def compute_w_ee(N, a, patterns):
    """
    Compute synaptic weights for excitatory-excitatory.
    """
    c = 2 / (a * (1 - a))
    w_ee = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            w_ee[i, j] = c / N * np.dot(patterns[:, i], patterns[:, j])
    return w_ee

def compute_w_ie(K, N_I):
    """
    Compute synaptic weights for excitatory-inhibitory.
    """
    return np.ones((N_I))/K

def compute_w_ei(N_I, a, patterns):
    """
    Compute synaptic weights for inhibitory-excitatory.
    """
    c = 2 / (a * (1 - a))
    w_ei = c*a/N_I * np.sum(patterns, axis=0)
    return w_ei

def compute_inputs_exc(N, N_I, w_ee, w_ei, sigma, sigma_I, selected_exc):
    """
    Compute the state of the network.
    Each inhibitry neuron output maps back to its pre-synaptic excitatory neurons.
    """
    sigma_I_expanded = np.zeros((N, N_I))
    #print(selected_exc)
    for i in range(N_I):
        sigma_I_expanded[selected_exc[:,i],i] = sigma_I[i]
    sigma_I_expanded = np.sum(sigma_I_expanded, axis=1)
    return np.dot(w_ee, sigma) - np.dot(w_ei, sigma_I_expanded)

def compute_inputs_inh(K, N_I, w_ie, sigma):
    """
    Compute the state of the network.
    We store the selected excitatory neurons for each inhibitory neuron.
    """
    h_inh = np.zeros(N_I)
    selected_exc = np.zeros((K, N_I), dtype=int)
    for i in range(N_I):
        selected_exc[:,i] = np.random.choice(N, size=K, replace=False)
        input_exc = sigma[selected_exc[:,i]]
        h_inh[i] = w_ie[i]*np.sum(input_exc)
    return h_inh, selected_exc

def compute_state(s, theta):
    """
    Compute the state of the network.
    """
    return np.tanh(s-theta)


In [55]:
def update_network(N, N_I, sigma_E, sigma_I, W_EE, W_EI, W_IE, selected_neurons, theta):
    """
    Update the network synchronously.
    """
    inputs_E = compute_inputs_exc(N, N_I, W_EE, sigma_E, W_EI, sigma_I, selected_neurons)
    inputs_I, selected_neurons = compute_inputs_inh(K, N_I, W_IE, sigma_E)

    # Update states based on inputs
    #sigma_E = np.array([stochastic_spike_variable_exc(input - theta) for input in inputs_E]) # fix this
    #sigma_I = np.array([stochastic_spike_variable_inh(input - theta) for input in inputs_I]) # fix this

    state_E = compute_state(inputs_E, theta)
    #print(inputs_I)
    state_I = compute_state(inputs_I, theta)
    #print(state_I)
    return state_E, state_I, selected_neurons

def simulate_network(N, N_I, M, K, a, theta, num_steps, update_type='synchronous'):
    """
    Simulate the network dynamics.
    """
    patterns = generate_low_activity_patterns(M, N, a)
    W_EE = compute_w_ee(N, a, patterns)
    W_EI = compute_w_ei(N_I, a, patterns)
    W_IE = compute_w_ie(K, N_I)

    # Initial random states
    state_E = np.random.choice([0, 1], size=N, p=[1-a, a])
    state_I = np.random.choice([0, 1], size=N_I, p=[1-a, a])

    sigma_E = stochastic_spike_variable_exc(state_E)
    sigma_I = stochastic_spike_variable_inh(state_I)

    selected_neurons = np.zeros((K, N_I), dtype=int)

    for step in range(num_steps):
        if update_type == 'synchronous':
            state_E, state_I, selected_neurons = update_network(N, N_I, sigma_E, sigma_I, W_EE, W_EI, W_IE, selected_neurons, theta)
            sigma_E = stochastic_spike_variable_exc(state_E)
            #print(state_I)
            sigma_I = stochastic_spike_variable_inh(state_I)
            
        else:
            # Asynchronous update: Update one neuron at a time
            for i in range(N):
                sigma_E[i] = stochastic_spike_variable_exc(compute_state(compute_inputs_exc(N, N_I, W_EE[i,:], W_EI[i], sigma_E, sigma_I, selected_neurons),theta))[i]
            for k in range(N_I):
                h_inh, selected_neurons = compute_inputs_inh(K, N_I, W_IE, sigma_E)
                sigma_I[k] = stochastic_spike_variable_inh(compute_state(h_inh,theta))[k]

    return sigma_E, sigma_I

# Parameters
N = 300  # Number of excitatory neurons
N_I = 80 # Number of inhibitory neurons
M = 10   # Number of patterns
K = 60   # Connections from excitatory to inhibitory
a = 0.1  # Activity level
theta = 0# Threshold
num_steps = 100

# Run the simulation
#states_sync = simulate_network(N, N_I, M, K, a, theta, num_steps, update_type='synchronous')
states_async = simulate_network(N, N_I, M, K, a, theta, num_steps, update_type='asynchronous')
